In [1]:
# Copyright 2024 ByteDance and/or its affiliates.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import os
os.chdir("..")
os.getcwd()

'/root/code/Protenix'

In [2]:
from IPython.display import display
import ipywidgets as widgets
import json
from copy import deepcopy
import os


from protenix.web_service.colab_request_parser import RequestParser
from protenix.web_service.viewer import ProtenixInputViewer
from protenix.web_service.prediction_visualization import (
    plot_confidence_measures_from_pred,
    plot_contact_maps_from_pred,
    PredictionLoader,
    plot_3d,
)

# Input - information about input complex

## Add global information
- **name**: System name
- **use_msa**: Whether to use MSA information during inference.
- **atom_confidence**: Whether to return the confidence of each atom during inference.
## Add Entity
For each entity, you need to select the type, set Copies and Sequence or SMILES or CCD information.
- Click **+Dna/Rna/Protein** to add Dna/Rna/Protein sequence.
- Click **+Ligand(SMILES)** to add Ligand SMILES.
- Click **+Ligand/Ion CCD** to add Ligand/Ion CCD.
### Add Modification for Dna/Rna/Protein
Each Dna/Rna/Protein entity can have multiple modifications. Modification information includes modification type and position.
- Click **+modification** to add a modification.
## Add covalent bond
Each covalent bond contains left/right entity, left/right pos and left/right atom information.
- Click **+convalent_bond** to add a covalent bond.
## Add model inference information
- **N_sample**: The number of samples generated by one seed (diffusion model).
- **N_step**: diffusion model step.
- **N_cycle**: The number of cycles during backbone inference.
- **seeds**: Random seeds.
- **version**: v1/v2/v3/v4/v5. Currently only v1 is supported.

In [3]:
"""
Additional user specified arguments
    - need_all_atom_confidence (bool), if saving per-atom confidence metrics (such as atom_plddt)
    - 
"""

viewer = ProtenixInputViewer()
display(viewer)

ProtenixInputViewer(children=(HBox(children=(Text(value='', description='name', placeholder='name'), Checkbox(…

# Input - information about results saving

In [4]:
save_dir = "./demo"
if os.path.exists(save_dir):
    print(
        (
            f"[WARNING]: Results will be saved to an existing path:\n{save_dir}.\n"
            "Please verify that this is the intended location before proceeding."
        )
    )
os.makedirs(save_dir, exist_ok=True)

# Inference code body

In [6]:
input_dict = viewer.get_result()
output = json.dumps(input_dict, ensure_ascii=False, indent=4)
with open(input_dict_path:=f"{save_dir}/input.json", "w") as fid:
    fid.write(output)

print(f"Input information saved to:\n{input_dict_path}")
print("Please verify the input information below:")
! cat $input_dict_path

Input information saved to:
./demo/input.json
Please verify the input information below:
{
    "name": "test",
    "use_msa": true,
    "atom_confidence": false,
    "sequences": [],
    "covalent_bonds": [],
    "model_seeds": [
        1,
        2,
        3,
        4,
        5
    ],
    "model_version": "v1",
    "N_sample": 5,
    "N_step": 200,
    "N_cycle": 10
}

In [7]:
parser = RequestParser(
    request_json_path=input_dict_path, request_dir=save_dir
)
parser.launch()

 https://af3-dev.tos-cn-beijing.volces.com/release_data/components.v20240608.cif...
Succeeded downloading from https://af3-dev.tos-cn-beijing.volces.com/release_data/components.v20240608.cif.
Saved to /af3-dev/release_data/components.v20240608.cif.
 https://af3-dev.tos-cn-beijing.volces.com/release_data/components.v20240608.cif.rdkit_mol.pkl...
Succeeded downloading from https://af3-dev.tos-cn-beijing.volces.com/release_data/components.v20240608.cif.rdkit_mol.pkl.
Saved to /af3-dev/release_data/components.v20240608.cif.rdkit_mol.pkl.


AttributeError: 'NoneType' object has no attribute 'chain_id'

# Visualize predicted structures

In [ ]:
seed = 1  # which seed do you want to visualize?

job_name = input_dict["name"]
pred_base_path = f"{save_dir}/{job_name}/seed_{seed}/predictions"
pred_loader = PredictionLoader(pred_base_path)

### Visualizing contact maps

In [ ]:
# Visualizing contact maps

# String input widget
rep_atom = widgets.Text(value="CA", description="Rep Atom", disabled=False)

# Float input widget
threshold = widgets.FloatSlider(
    value=10,
    min=1,
    max=20.0,
    step=0.1,
    description="Threshold",
    continuous_update=False,
)


def plot_wrapper(threshold: float, rep_atom: str):
    plot_contact_maps_from_pred(
        preds=pred_loader.preds,
        fnames=pred_loader.fnames,
        threshold=threshold,
        rep_atom=rep_atom,
    )


interactive_plot = widgets.interactive(
    plot_wrapper, threshold=threshold, rep_atom=rep_atom
)

display(interactive_plot)

### Visualizing confidence metrics

In [ ]:
# Visualizing confidence metrics
if not pred_loader.full_confidence_data:
    print("All-atom confidence metrics not saved.")
else:
    bool_option = widgets.Checkbox(value=True, description="Print global metrics")

    def plot_wrapper(show_global_confidence: bool):
        plot_confidence_measures_from_pred(
            full_confidence=pred_loader.full_confidence_data,
            summary_confidence=pred_loader.summary_confidence_data,
            fnames=pred_loader.fnames,
            show_global_confidence=show_global_confidence,
        )

    interactive_plot = widgets.interactive(
        plot_wrapper, show_global_confidence=bool_option
    )

    display(interactive_plot)

### Visualizing 3d structure

In [ ]:
# pred_id: the i-th prediction from the model
pred_id = 0
plot_3d(
    pred_id=pred_id,
    pred_loader=pred_loader,
    show_sidechains=False,
    show_mainchains=False,
    color="rainbow",
)

In [ ]:
if not pred_loader.full_confidence_data:
    plot_3d_func = lambda *args, **kwargs: None
    print("All-atom confidence metrics not saved. Hence plotting by pLDDT disabled.")
else:
    plot_3d_func = deepcopy(plot_3d)
    pass

In [ ]:
plot_3d_func(
    pred_id=pred_id,
    pred_loader=pred_loader,
    show_sidechains=False,
    show_mainchains=False,
    color="pLDDT",
)